## Decision Tree V1
- all datetime feature
- random cross validation

**Hyperparameter for tweeking:**
- maximale Baumtiefe
- Mindestanzahl an Daten pro Blatt
- Mindestanzahl an Daten pro Knoten
- Features

**Regularisierung:**
- Pruning

**Metriken:**
- mean_absolute_error
- mean_squared_error
- median_absolute_error

#### **_PREPARATION_**

In [1]:
# GET ALL THE JSONS INTO ONE DATAFRAME
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import json
import glob

In [2]:
# Set the search path for files (assuming the directory is relative to the current script)
file_path_mc124 = os.path.join("..", "mc124_data", "*.json")
files = glob.glob(file_path_mc124)

# Create empty list to store dataframes
li_all_files = []

# Loop through list of files and read each one into a dataframe and append to list
for f in files:
    # Read in json
    temp_df = pd.read_json(f)
    # Append df to list
    li_all_files.append(temp_df)

# Optionally concatenate all dataframes into one if needed
if li_all_files:
    combined_df = pd.concat(li_all_files)
    print(f'Combined dataframe shape: {combined_df.shape}')
else:
    print('No dataframes were created.')

Combined dataframe shape: (542555, 6)


In [3]:
combined_df.sample()

,datetime,station,core,component,period,value
3,2021-02-28 23:00:00+01:00,mc124,no,no_1h,1h,28.0


In [4]:
# FILTER BY PARTICLE AND ONLY KEEP THE DATETIME, STATION, PERIOD AND VALUE FEATURE SINCE THE REST ARE CONSTANT INFORMATION (station, core, component, period)
df_reduced = combined_df[['datetime', 'station', 'core', 'value']]
df_reduced.sample(3)

# CUT OFF THE TIMEZONE INFORMATION FROM THE DATETIME TO AVOID CONVERSION ISSUES DUE TO TIME CHANGE IN MARCH AND OCTOBER
df_reduced.loc[:, 'datetime'] = df_reduced['datetime'].astype(str).str.slice(0, 19)
#df_pm10_reduced.loc[:, 'datetime'] = pd.to_datetime(df_pm10_reduced['datetime'], format='mixed')
df_reduced['datetime'] = pd.to_datetime(df_reduced['datetime'], format='mixed')
df_reduced.loc[:, 'datetime'] = df_reduced['datetime'].dt.tz_localize(None)
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 542555 entries, 0 to 3654
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   datetime  542555 non-null  datetime64[ns]
 1   station   542555 non-null  object        
 2   core      542555 non-null  object        
 3   value     539422 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 20.7+ MB


C:\Users\a_n_n\AppData\Local\Temp\ipykernel_4764\2472174257.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced['datetime'] = pd.to_datetime(df_reduced['datetime'], format='mixed')


In [5]:
df_reduced.sample(3)

,datetime,station,core,value
1790,2011-12-07 03:00:00,mc124,nox,37.0
1231,2009-05-14 21:00:00,mc124,no,6.0
940,2022-10-24 03:00:00,mc124,pm10,16.0


In [6]:
#use loc to add it to every for every row
df_reduced['hour'] = df_reduced['datetime'].dt.strftime('%H')  # Hour (00-23)
df_reduced['day'] = df_reduced['datetime'].dt.strftime('%d')  # Day of the month (01-31)
df_reduced['month'] = df_reduced['datetime'].dt.strftime('%m')  # Month (01-12)
df_reduced['year'] = df_reduced['datetime'].dt.strftime('%Y')  # Month (01-12)
df_reduced.sample(3)

C:\Users\a_n_n\AppData\Local\Temp\ipykernel_4764\493841492.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced['hour'] = df_reduced['datetime'].dt.strftime('%H')  # Hour (00-23)


,datetime,station,core,value,hour,day,month,year
2212,2024-04-12 13:00:00,mc124,no2,35.0,13,12,04,2024
1393,2022-11-19 09:00:00,mc124,no,90.0,09,19,11,2022
883,2024-05-24 02:00:00,mc124,no,2.0,02,24,05,2024


In [7]:
# add day of the week to dataframe
import calendar

days = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df_daytime = df_reduced
# convert the 'day', 'month', and 'year' columns to integers
df_daytime['day'] = df_reduced['day'].astype(int)
df_daytime['month'] = df_reduced['month'].astype(int)
df_daytime['year'] = df_reduced['year'].astype(int)

# function to determine the day of the week
def get_day_of_week(row):
    return calendar.weekday(row['year'], row['month'], row['day'])

# function to determine if day is weekday or weekend 
def is_weekend(day_number):
    return 1 if day_number >= 5 else 0
    #day_number >= 5 ? 1 : 0
    
# apply the functions to create the new columns
df_daytime['day_of_week'] = df_daytime.apply(get_day_of_week, axis=1)
df_daytime['is_weekend'] = df_daytime['day_of_week'].apply(is_weekend)

df_daytime.rename(columns={'value': 'pm10_value'}, inplace=True)

df_daytime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 542555 entries, 0 to 3654
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   datetime     542555 non-null  datetime64[ns]
 1   station      542555 non-null  object        
 2   core         542555 non-null  object        
 3   pm10_value   539422 non-null  float64       
 4   hour         542555 non-null  object        
 5   day          542555 non-null  int32         
 6   month        542555 non-null  int32         
 7   year         542555 non-null  int32         
 8   day_of_week  542555 non-null  int64         
 9   is_weekend   542555 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(3)
memory usage: 39.3+ MB


In [8]:
df_daytime.dropna(inplace=True)
df_daytime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 539422 entries, 0 to 3654
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   datetime     539422 non-null  datetime64[ns]
 1   station      539422 non-null  object        
 2   core         539422 non-null  object        
 3   pm10_value   539422 non-null  float64       
 4   hour         539422 non-null  object        
 5   day          539422 non-null  int32         
 6   month        539422 non-null  int32         
 7   year         539422 non-null  int32         
 8   day_of_week  539422 non-null  int64         
 9   is_weekend   539422 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(3)
memory usage: 39.1+ MB


#### **_ACTUAL MODEL TRAINING_**


#### Part 1: create decision tree on everything

In [35]:
# create features and to be predicted value
y = df_daytime['pm10_value']
X = df_daytime[['hour', 'year', 'day_of_week', 'is_weekend']]

# SCALING?

In [36]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree

In [37]:
dt = DecisionTreeRegressor(max_depth=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=1)
dt.fit(X_train, y_train)
dt.score(X_train, y_train)

0.1457118119110632

In [38]:
dt.score(X_test, y_test)

0.14182334982175882

In [39]:
dt = DecisionTreeRegressor(max_depth=2000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=1)
dt.fit(X_train, y_train)
dt.score(X_train, y_train)

0.24453117558268656

In [25]:
dt.score(X_test, y_test)

0.11437126086933513

In [48]:
# cross validate
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error

# scores = cross_validate(knn, data_scaled, iris.target, return_train_score=True, cv=5)
# scores
# train_scores = scores["train_score"]
# test_scores = scores["test_score"]
# train_scores.mean()
# test_scores.mean()
# test_scores.std()

#-------------------------------------------------------------------

# Create a DecisionTreeRegressor
decision_tree = DecisionTreeRegressor()

# Define your scoring functions
scorers = {
    'mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False),
    'mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False),
    'median_absolute_error': make_scorer(median_absolute_error, greater_is_better=False)
}

# Perform cross-validation
scores = cross_validate(decision_tree, X, y, cv=5, scoring=scorers, return_train_score=True)

# Print the results
print("Cross-validation scores:")
for scorer, score in scores.items():
    print(f"{scorer}: {np.mean(score):.2f} (+/- {np.std(score):.2f})")

Cross-validation scores:
fit_time: 0.51 (+/- 0.12)
score_time: 0.05 (+/- 0.01)
test_mean_absolute_error: -34.19 (+/- 9.38)
train_mean_absolute_error: -32.99 (+/- 2.68)
test_mean_squared_error: -2964.36 (+/- 1580.67)
train_mean_squared_error: -2821.03 (+/- 378.08)
test_median_absolute_error: -22.86 (+/- 5.00)
train_median_absolute_error: -20.71 (+/- 1.94)


In [45]:
scores

{'fit_time': array([0.41242266, 0.48479939, 0.5263772 , 0.63439274, 0.61335135]),
 'score_time': array([0.04782224, 0.05198812, 0.05761933, 0.07002497, 0.06286621]),
 'test_mean_absolute_error': array([-45.68906005, -43.23609553, -34.32319257, -26.38275244,
        -21.33665478]),
 'test_mean_squared_error': array([-4647.09647197, -4862.78049234, -2789.90800272, -1595.44813097,
         -926.55201736]),
 'test_median_absolute_error': array([-30.82962963, -25.11111111, -23.09137056, -18.53066667,
        -16.71428571])}

## TODO: REMOVE CONSISTENTLY HEAVY OUTLIER
- EASTER
- CHRISTMAS
-> take one year and analyze month for month (daily data)
-> additional column: crazy_holiday


In [47]:
# average pm10 value for comparison to the error rates
average_pm10 = df_daytime['pm10_value'].mean()
print(f"Average PM10 value: {average_pm10:.2f}")

Average PM10 value: 51.14
